# **Projeto: Quais fatores mais influenciam no desempenho de um aluno no ENEM e como podemos ajudá-lo a melhorar sua performance?**

**Integrantes:** Hellen Cristine Silva Rosa (RA00319076), João Victor Porto (RA00311353), Laura Gabriel Murayama (RA00319321), Maria Eduarda Bonel Iribarnegaray (RA00318891), Vinícius Ferreira de Mendonça (RA00319760), Vitória de Fátima Teixeira (RA00320578)

### **Importando bibliotecas e definindo funções importantes**

In [ ]:
SEED = 3

# loading environment variables
from dotenv import load_dotenv
load_dotenv()

# importing sklearn accelerator
from sklearnex import patch_sklearn, unpatch_sklearn

# importing libraries
import os
import re
import psycopg2
import pandas as pd
import numpy as np
import joblib
import skops.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

# importing sklearn functions
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, precision_recall_fscore_support, ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

In [ ]:
# establishing connection with DB
class db_connection():
    '''
    Instantiates a connection with the database.
    '''
    def __init__(self):
        self.DB_URI = os.environ.get('DB_URI')
         
    def __enter__(self):
        self.connection = psycopg2.connect(self.DB_URI)
        return self.connection
    
    def __exit__(self, *args):
        self.connection.close()

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.aggregated_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''
    
    cursor.execute(query)
    aggregated_data = cursor.fetchall()
    aggregated_columns = tuple(desc[0] for desc in cursor.description)


aggregated_df = pd.DataFrame(aggregated_data, columns=aggregated_columns)
aggregated_df

In [ ]:
aggregated_df = pd.read_csv('../aggregated_df.csv')
aggregated_df = aggregated_df.replace(np.nan, None)
aggregated_df

### **REQ#01: Utilizar um ou mais datasets para o treinamento dos classificadores**

Utilizaremos os microdados do ENEM 2022, [disponibilizados pelo INEP](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem).

### **REQ#02: Análise Exploratória**

#### Verificando nulos

In [ ]:
updated_aggregated_df = aggregated_df.copy()

In [ ]:
updated_aggregated_df

In [ ]:
updated_aggregated_df = updated_aggregated_df.dropna(
    subset=['NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_SG_UF_ESC', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_Q005', 'PARTICIPANTE_Q006', 'HABESTUDO_Q001', 'HABESTUDO_Q002', 'HABESTUDO_Q003', 'HABESTUDO_Q004', 'HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q025A', 'HABESTUDO_Q025B', 'HABESTUDO_Q025C', 'HABESTUDO_Q025D', 'HABESTUDO_Q025E', 'HABESTUDO_Q025F', 'HABESTUDO_Q026A', 'HABESTUDO_Q026B', 'HABESTUDO_Q026C', 'HABESTUDO_Q026D', 'HABESTUDO_Q026E', 'HABESTUDO_Q026F', 'HABESTUDO_Q026G', 'HABESTUDO_Q026H', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R', 'HABESTUDO_Q029', 'HABESTUDO_Q030A', 'HABESTUDO_Q030B', 'HABESTUDO_Q030C', 'HABESTUDO_Q030D', 'HABESTUDO_Q030E', 'HABESTUDO_Q030F', 'HABESTUDO_Q030G', 'HABESTUDO_Q031', 'HABESTUDO_Q032A', 'HABESTUDO_Q032B', 'HABESTUDO_Q032C', 'HABESTUDO_Q032D', 'HABESTUDO_Q032E', 'HABESTUDO_Q032F', 'HABESTUDO_Q032G', 'HABESTUDO_Q033A', 'HABESTUDO_Q033B', 'HABESTUDO_Q033C', 'HABESTUDO_Q033D', 'HABESTUDO_Q033E', 'HABESTUDO_Q033F', 'HABESTUDO_Q033G', 'HABESTUDO_Q033H', 'HABESTUDO_Q033I', 'HABESTUDO_Q033J', 'HABESTUDO_Q034']
).reset_index(drop=True)

updated_aggregated_df

In [ ]:
updated_aggregated_df_isna = updated_aggregated_df.isna().sum()
updated_aggregated_df_isna[updated_aggregated_df_isna != 0]

#### Criando novas variáveis

In [ ]:
def get_regiao_uf(vetor_uf: pd.Series) -> pd.Series:
    '''
    Informa a região correspondente ao estado a partir de sua sigla.

    :params:
    - vetor_uf: UF do candidato (MICRODADOS_SG_UF_ESC).

    Returns a Series.
    '''

    regioes_list = []

    for uf in vetor_uf:
        regiao_uf_dict = {
            'AM': 'Norte',
            'RR': 'Norte',
            'AP': 'Norte',
            'PA': 'Norte',
            'TO': 'Norte',
            'RO': 'Norte',
            'AC': 'Norte',
            'MA': 'Nordeste',
            'PI': 'Nordeste',
            'CE': 'Nordeste',
            'RN': 'Nordeste',
            'PE': 'Nordeste',
            'PB': 'Nordeste',
            'SE': 'Nordeste',
            'AL': 'Nordeste',
            'BA': 'Nordeste',
            'MT': 'Centro-Oeste',
            'MS': 'Centro-Oeste',
            'GO': 'Centro-Oeste',
            'DF': 'Centro-Oeste',
            'SP': 'Sudeste',
            'RJ': 'Sudeste',
            'ES': 'Sudeste',
            'MG': 'Sudeste',
            'PR': 'Sul',
            'RS': 'Sul',
            'SC': 'Sul',
        }

        regiao = regiao_uf_dict[uf]
        regioes_list.append(regiao)

    return pd.Series(regioes_list)

In [ ]:
def get_nota_conceito(vetor_nota_media: pd.Series, porcentagem_quartis: float) -> pd.Series:
    '''
    Converte as notas médias dos participantes em conceitos a partir da divisão dessas notas em quartis.

    :params:
    - vetor_nota_media: nota média do candidato (MICRODADOS_NU_NOTA_MEDIA).
    - porcentagem_quartis: tamanho de cada um dos quartis a ser didivido.

    Returns a Series.
    '''

    quantile_percentages = tuple(map(lambda x: round(x, 2), np.arange(0.0, (1.0 + porcentagem_quartis), porcentagem_quartis)))

    quantiles = tuple(map(lambda x: round(vetor_nota_media.quantile(x), 2), quantile_percentages))

    if quantiles[0] > vetor_nota_media.min():
        quantiles[0] = vetor_nota_media.min()
    if quantiles[-1] < vetor_nota_media.max():
        quantiles[-1] = vetor_nota_media.max()

    quantiles_intervals = tuple(zip(quantiles, quantiles[1:]))


    conceito_list = []

    for nota_media in vetor_nota_media:
        for interval_index, interval in enumerate(quantiles_intervals):
            if (interval[0] <= nota_media) and (nota_media <= interval[1]):
                conceito = interval_index
                conceito_list.append(conceito)
                break


    return pd.Series(conceito_list)

In [ ]:
def count_acertos(area_prova: str, vetor_respostas: pd.Series, vetor_gabaritos: pd.Series, vetor_linguas: pd.Series | None = None) -> pd.Series:
    '''
    Conta os acertos de determinada prova.

    :params:
    - area_prova: a qual área corresponde a prova.
    - vetor_respostas: vetor de respostas do candidato.
    - vetor_gabaritos: vetor de gabaritos dada a prova do candidato.
    - vetor_linguas: caso 'area_prova' seja 'LC', a lingua deve ser indicada, sendo 0 para 'inglês' e 1 para 'espanhol'.

    Returns a Series.
    '''

    if (area_prova == 'LC') and (isinstance(vetor_linguas, type(None))):
        raise ValueError('vetor_linguas must be defined for a \'LC\' area_prova.')


    def update_gabarito_with_language(lingua: int, gabarito: str):
        if not isinstance(lingua, int):
            raise ValueError('\'lingua\' is not defined')
        elif lingua == 0:
            updated_gabarito = gabarito[:5] + gabarito[10:]
        elif lingua == 1:
            updated_gabarito = gabarito[5:]
        return updated_gabarito
    

    def check_answers_on_gabarito(acertos_list: list, respostas: str, gabarito: str, lingua: int | None = None):
        if not isinstance(respostas, str):
            acertos_list.append(None)
        else:
            if area_prova == 'LC':
                gabarito = update_gabarito_with_language(lingua, gabarito)

            acertos = 0
            for r, g in zip(respostas, gabarito):
                if r == g:
                    acertos += 1
            acertos_list.append(acertos)
    
    
    acertos_list = []
    if area_prova == 'LC':
        for respostas, gabarito, lingua in zip(vetor_respostas, vetor_gabaritos, vetor_linguas):
            check_answers_on_gabarito(acertos_list, respostas, gabarito, lingua)
    else:
        for respostas, gabarito in zip(vetor_respostas, vetor_gabaritos):
            check_answers_on_gabarito(acertos_list, respostas, gabarito)
    

    return pd.Series(acertos_list)

In [ ]:
def get_mediana_renda_per_capita(vetor_renda_total: pd.Series, vetor_tamanho_grupo_familiar: pd.Series) -> pd.Series:
    '''
    Calcula a renda per capita do grupo familiar.

    :params:
    - vetor_renda_total: item de resposta da renda total (MICRODADOS_Q006).
    - vetor_tamanho_grupo_familiar: item de resposta do tamanho do grupo_familiar (MICRODADOS_Q005).

    Returns a Series.
    '''

    renda_per_capita_list = []

    for renda, tamanho_grupo_familiar in zip(vetor_renda_total, vetor_tamanho_grupo_familiar):
        renda_correspondence_dict = {
            'A': (0.00, 0.00),
            'B': (0.00, 1212.00),
            'C': (1212.01, 1818.00),
            'D': (1818.01, 2424.00),
            'E': (2424.01, 3030.00),
            'F': (3030.01, 3636.00),
            'G': (3636.01, 4848.00),
            'H': (4848.01, 6060.00),
            'I': (6060.01, 7272.00),
            'J': (7272.01, 8484.00),
            'K': (8484.01, 9696.00),
            'L': (9696.01, 10908.00),
            'M': (10908.01, 12120.00),
            'N': (12120.01, 14544.00),
            'O': (14544.01, 18180.00),
            'P': (18180.01, 24240.00),
            'Q': (24240.01, float('inf')),
        }
        intervalo_renda = renda_correspondence_dict[renda]

        if intervalo_renda:
            intervalo_renda_per_capita = tuple([amount/tamanho_grupo_familiar for amount in intervalo_renda])
            mediana_renda_per_capita = np.median(intervalo_renda_per_capita)
            mediana_renda_per_capita = round(mediana_renda_per_capita, 2)
            renda_per_capita_list.append(mediana_renda_per_capita)
        else:
            renda_per_capita_list.append(None)

    return pd.Series(renda_per_capita_list)

In [ ]:
'NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_SG_UF_ESC', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_Q005', 'PARTICIPANTE_Q006', 'HABESTUDO_Q001', 'HABESTUDO_Q002', 'HABESTUDO_Q003', 'HABESTUDO_Q004', 'HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q025A', 'HABESTUDO_Q025B', 'HABESTUDO_Q025C', 'HABESTUDO_Q025D', 'HABESTUDO_Q025E', 'HABESTUDO_Q025F', 'HABESTUDO_Q026A', 'HABESTUDO_Q026B', 'HABESTUDO_Q026C', 'HABESTUDO_Q026D', 'HABESTUDO_Q026E', 'HABESTUDO_Q026F', 'HABESTUDO_Q026G', 'HABESTUDO_Q026H', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R', 'HABESTUDO_Q029', 'HABESTUDO_Q030A', 'HABESTUDO_Q030B', 'HABESTUDO_Q030C', 'HABESTUDO_Q030D', 'HABESTUDO_Q030E', 'HABESTUDO_Q030F', 'HABESTUDO_Q030G', 'HABESTUDO_Q031', 'HABESTUDO_Q032A', 'HABESTUDO_Q032B', 'HABESTUDO_Q032C', 'HABESTUDO_Q032D', 'HABESTUDO_Q032E', 'HABESTUDO_Q032F', 'HABESTUDO_Q032G', 'HABESTUDO_Q033A', 'HABESTUDO_Q033B', 'HABESTUDO_Q033C', 'HABESTUDO_Q033D', 'HABESTUDO_Q033E', 'HABESTUDO_Q033F', 'HABESTUDO_Q033G', 'HABESTUDO_Q033H', 'HABESTUDO_Q033I', 'HABESTUDO_Q033J', 'HABESTUDO_Q034'

In [ ]:
updated_aggregated_df.loc[:, 'PARTICIPANTE_REGIAO_ESCOLA'] = get_regiao_uf(updated_aggregated_df['PARTICIPANTE_SG_UF_ESC'])

updated_aggregated_df.loc[:, 'NOTAS_NU_NOTA_MEDIA'] = pd.Series(map(lambda x: round(x, 2), (updated_aggregated_df['NOTAS_NU_NOTA_CN'] + updated_aggregated_df['NOTAS_NU_NOTA_CH'] + updated_aggregated_df['NOTAS_NU_NOTA_LC'] + updated_aggregated_df['NOTAS_NU_NOTA_MT'] + updated_aggregated_df['NOTAS_NU_NOTA_REDACAO']) / 5))

updated_aggregated_df.loc[:, 'NOTAS_NU_NOTA_CONCEITO'] = get_nota_conceito(updated_aggregated_df['NOTAS_NU_NOTA_MEDIA'], 0.25)

updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_CN'] = count_acertos('CN', updated_aggregated_df['NOTAS_TX_RESPOSTAS_CN'], updated_aggregated_df['NOTAS_TX_GABARITO_CN'])
updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_CH'] = count_acertos('CH', updated_aggregated_df['NOTAS_TX_RESPOSTAS_CH'], updated_aggregated_df['NOTAS_TX_GABARITO_CH'])
updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_LC'] = count_acertos('LC', updated_aggregated_df['NOTAS_TX_RESPOSTAS_LC'], updated_aggregated_df['NOTAS_TX_GABARITO_LC'], updated_aggregated_df['NOTAS_TP_LINGUA'])
updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_MT'] = count_acertos('MT', updated_aggregated_df['NOTAS_TX_RESPOSTAS_MT'], updated_aggregated_df['NOTAS_TX_GABARITO_MT'])

updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_TOTAL'] = updated_aggregated_df['NOTAS_NU_ACERTOS_CN'] + updated_aggregated_df['NOTAS_NU_ACERTOS_CH'] + updated_aggregated_df['NOTAS_NU_ACERTOS_LC'] + updated_aggregated_df['NOTAS_NU_ACERTOS_MT']
updated_aggregated_df.loc[:, 'NOTAS_NU_ACERTOS_MEDIO'] = round(updated_aggregated_df['NOTAS_NU_ACERTOS_TOTAL'] / 4, 2)

updated_aggregated_df.loc[:, 'PARTICIPANTE_RENDA_PER_CAPITA'] = get_mediana_renda_per_capita(updated_aggregated_df['PARTICIPANTE_Q006'], updated_aggregated_df['PARTICIPANTE_Q005'])


updated_aggregated_df

In [ ]:
def get_hab_estudo_category_values(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Une as colunas de respostas dos hábitos de estudo por categoria, normalizando as respostas.

    :params:
    - df: DataFrame com todos os dados para substituição.

    Returns a DataFrame.
    '''

    # colocar variáveis em ordem crescente de dificuldade/falta de estudo (positividade)
    # em caso de sim/não, utilizar 'sim' como maior
    hab_estudo_correspondence_dict = {
        'HABESTUDO_ST_MAT_PERC_APR': {
            'HABESTUDO_Q001': ('E', 'D', 'C', 'B', 'A'),
            'HABESTUDO_Q002': ('C', 'B', 'A'),
            'HABESTUDO_Q003': ('E', 'D', 'C', 'B', 'A'),
            'HABESTUDO_Q004': ('F', 'E', 'D', 'C', 'B', 'A'),
            },
        'HABESTUDO_GES_TEMP_PLAN_EST': {
            'HABESTUDO_Q005': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q006': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q007': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q008': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q021': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q022': ('A', 'B', 'C', 'D'),
            },
        'HABESTUDO_PRAT_EST_PES': {
            'HABESTUDO_Q009': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q010': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q011': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q012': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q013': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q014': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q015': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q016': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q017': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q018': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q019': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q020': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q023': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q024': ('A', 'B', 'C', 'D'),
            },
        'HABESTUDO_TECN_TP_ACES': {
            'HABESTUDO_Q025A': ('B', 'A'),
            'HABESTUDO_Q025B': ('B', 'A'),
            'HABESTUDO_Q025C': ('B', 'A'),
            'HABESTUDO_Q025D': ('B', 'A'),
            'HABESTUDO_Q025E': ('B', 'A'),
            'HABESTUDO_Q025F': ('B', 'A'),
            'HABESTUDO_Q026A': ('B', 'A'),
            'HABESTUDO_Q026B': ('B', 'A'),
            'HABESTUDO_Q026C': ('B', 'A'),
            'HABESTUDO_Q026D': ('B', 'A'),
            'HABESTUDO_Q026E': ('B', 'A'),
            'HABESTUDO_Q026F': ('B', 'A'),
            'HABESTUDO_Q026G': ('B', 'A'),
            'HABESTUDO_Q026H': ('B', 'A'),
            },
        'HABESTUDO_PROB_ROT_EST': {
            'HABESTUDO_Q027': ('B', 'A'),
            'HABESTUDO_Q028A': ('B', 'A'),
            'HABESTUDO_Q028B': ('B', 'A'),
            'HABESTUDO_Q028C': ('B', 'A'),
            'HABESTUDO_Q028D': ('B', 'A'),
            'HABESTUDO_Q028E': ('B', 'A'),
            'HABESTUDO_Q028F': ('B', 'A'),
            'HABESTUDO_Q028G': ('B', 'A'),
            'HABESTUDO_Q028H': ('B', 'A'),
            'HABESTUDO_Q028I': ('B', 'A'),
            'HABESTUDO_Q028J': ('B', 'A'),
            'HABESTUDO_Q028K': ('B', 'A'),
            'HABESTUDO_Q028L': ('B', 'A'),
            'HABESTUDO_Q028M': ('B', 'A'),
            'HABESTUDO_Q028N': ('B', 'A'),
            'HABESTUDO_Q028O': ('B', 'A'),
            'HABESTUDO_Q028P': ('B', 'A'),
            'HABESTUDO_Q028Q': ('B', 'A'),
            'HABESTUDO_Q028R': ('B', 'A'),
            },
        'HABESTUDO_DIF_INFR': {
            'HABESTUDO_Q029': ('B', 'A'),
            'HABESTUDO_Q030A': ('B', 'A'),
            'HABESTUDO_Q030B': ('B', 'A'),
            'HABESTUDO_Q030C': ('B', 'A'),
            'HABESTUDO_Q030D': ('B', 'A'),
            'HABESTUDO_Q030E': ('B', 'A'),
            'HABESTUDO_Q030F': ('B', 'A'),
            'HABESTUDO_Q030G': ('B', 'A'),
            },
        'HABESTUDO_AJUD_TERC': {
            'HABESTUDO_Q031': ('C', 'A', 'B'),
            'HABESTUDO_Q032A': ('B', 'A'),
            'HABESTUDO_Q032B': ('B', 'A'),
            'HABESTUDO_Q032C': ('B', 'A'),
            'HABESTUDO_Q032D': ('B', 'A'),
            'HABESTUDO_Q032E': ('B', 'A'),
            'HABESTUDO_Q032F': ('B', 'A'),
            'HABESTUDO_Q032G': ('B', 'A'),
            'HABESTUDO_Q033A': ('B', 'A'),
            'HABESTUDO_Q033B': ('B', 'A'),
            'HABESTUDO_Q033C': ('B', 'A'),
            'HABESTUDO_Q033D': ('B', 'A'),
            'HABESTUDO_Q033E': ('B', 'A'),
            'HABESTUDO_Q033F': ('B', 'A'),
            'HABESTUDO_Q033G': ('B', 'A'),
            'HABESTUDO_Q033H': ('B', 'A'),
            'HABESTUDO_Q033I': ('B', 'A'),
            'HABESTUDO_Q033J': ('B', 'A'),
            },
        'HABESTUDO_AVAL_PROP_EXP': {
            'HABESTUDO_Q034': ('A', 'B', 'C', 'D', 'E'),
            },
    }


    for column_name, column_questions_dict in hab_estudo_correspondence_dict.items():
        column_series_dict = {}

        for question_name, question_value_list in column_questions_dict.items():
            question_series = []
            
            for student_answer in df[question_name]:
                row_value = question_value_list.index(student_answer) / (len(question_value_list) - 1)
                row_value = round(row_value, 2)
                question_series.append(row_value)
            
            column_series_dict[question_name] = question_series

        column_series = pd.Series(zip(*column_series_dict.values()))
        column_series = column_series.apply(lambda values_tuple: sum(values_tuple))

        df.loc[:, column_name] = column_series


    return df

In [ ]:
updated_aggregated_df = get_hab_estudo_category_values(updated_aggregated_df)
updated_aggregated_df

#### Filtrando variáveis

In [ ]:
interest_variables = ['NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_REGIAO_ESCOLA', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_ACERTOS_TOTAL', 'NOTAS_NU_ACERTOS_MEDIO', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_RENDA_PER_CAPITA', 'HABESTUDO_ST_MAT_PERC_APR', 'HABESTUDO_GES_TEMP_PLAN_EST', 'HABESTUDO_PRAT_EST_PES', 'HABESTUDO_TECN_TP_ACES', 'HABESTUDO_PROB_ROT_EST', 'HABESTUDO_DIF_INFR', 'HABESTUDO_AJUD_TERC', 'HABESTUDO_AVAL_PROP_EXP']

filtered_aggregated_df = updated_aggregated_df.loc[:, interest_variables]
filtered_aggregated_df

#### Excluindo rendas per capita infinitas

In [ ]:
renda_inf_indexes = filtered_aggregated_df[filtered_aggregated_df['PARTICIPANTE_RENDA_PER_CAPITA'] == float('inf')].index

filtered_aggregated_df = filtered_aggregated_df.drop(index=renda_inf_indexes).reset_index(drop=True)
filtered_aggregated_df

#### Exportando dados localmente e para o DB

In [ ]:
filtered_aggregated_df.to_csv('../filtered_aggregated_df.csv', index=False)

In [ ]:
filtered_aggregated_data_creation_query = '''
CREATE TABLE projeto_enem.filtered_aggregated_data (
	"NU_INSCRICAO" BIGINT PRIMARY KEY,
	"PARTICIPANTE_TP_FAIXA_ETARIA" INTEGER,
	"PARTICIPANTE_TP_SEXO" VARCHAR(1),
	"PARTICIPANTE_TP_COR_RACA" INTEGER,
	"PARTICIPANTE_TP_ST_CONCLUSAO" INTEGER,
	"PARTICIPANTE_TP_ANO_CONCLUIU" INTEGER,
	"PARTICIPANTE_TP_ESCOLA" INTEGER,
	"PARTICIPANTE_IN_TREINEIRO" INTEGER,
	"PARTICIPANTE_REGIAO_ESCOLA" VARCHAR(12),
	"PARTICIPANTE_TP_PRESENCA_CN" INTEGER,
	"PARTICIPANTE_TP_PRESENCA_CH" INTEGER,
	"PARTICIPANTE_TP_PRESENCA_LC" INTEGER,
	"PARTICIPANTE_TP_PRESENCA_MT" INTEGER,
	"NOTAS_NU_NOTA_CN" FLOAT,
	"NOTAS_NU_NOTA_CH" FLOAT,
	"NOTAS_NU_NOTA_LC" FLOAT,
	"NOTAS_NU_NOTA_MT" FLOAT,
	"NOTAS_NU_NOTA_REDACAO" FLOAT,
	"PARTICIPANTE_TP_STATUS_REDACAO" FLOAT,
	"NOTAS_NU_NOTA_COMP1" FLOAT,
	"NOTAS_NU_NOTA_COMP2" FLOAT,
	"NOTAS_NU_NOTA_COMP3" FLOAT,
	"NOTAS_NU_NOTA_COMP4" FLOAT,
	"NOTAS_NU_NOTA_COMP5" FLOAT,
	"NOTAS_NU_NOTA_MEDIA" FLOAT,
    "NOTAS_NU_NOTA_CONCEITO" INTEGER,
	"NOTAS_NU_ACERTOS_CN" INTEGER,
	"NOTAS_NU_ACERTOS_CH" INTEGER,
	"NOTAS_NU_ACERTOS_LC" INTEGER,
	"NOTAS_NU_ACERTOS_MT" INTEGER,
	"NOTAS_NU_ACERTOS_TOTAL" INTEGER,
	"NOTAS_NU_ACERTOS_MEDIO" FLOAT,
	"PARTICIPANTE_TP_LINGUA" INTEGER,
	"PARTICIPANTE_RENDA_PER_CAPITA" FLOAT,
	"HABESTUDO_ST_MAT_PERC_APR" FLOAT,
	"HABESTUDO_GES_TEMP_PLAN_EST" FLOAT,
	"HABESTUDO_PRAT_EST_PES" FLOAT,
	"HABESTUDO_TECN_TP_ACES" FLOAT,
	"HABESTUDO_PROB_ROT_EST" FLOAT,
	"HABESTUDO_DIF_INFR" FLOAT,
	"HABESTUDO_AJUD_TERC" FLOAT,
	"HABESTUDO_AVAL_PROP_EXP" FLOAT
);
'''

filtered_aggregated_csv_query = '''
COPY projeto_enem.filtered_aggregated_data("NU_INSCRICAO", "PARTICIPANTE_TP_FAIXA_ETARIA", "PARTICIPANTE_TP_SEXO", "PARTICIPANTE_TP_COR_RACA", "PARTICIPANTE_TP_ST_CONCLUSAO", "PARTICIPANTE_TP_ANO_CONCLUIU", "PARTICIPANTE_TP_ESCOLA", "PARTICIPANTE_IN_TREINEIRO", "PARTICIPANTE_REGIAO_ESCOLA", "PARTICIPANTE_TP_PRESENCA_CN", "PARTICIPANTE_TP_PRESENCA_CH", "PARTICIPANTE_TP_PRESENCA_LC", "PARTICIPANTE_TP_PRESENCA_MT", "NOTAS_NU_NOTA_CN", "NOTAS_NU_NOTA_CH", "NOTAS_NU_NOTA_LC", "NOTAS_NU_NOTA_MT", "NOTAS_NU_NOTA_REDACAO", "PARTICIPANTE_TP_STATUS_REDACAO", "NOTAS_NU_NOTA_COMP1", "NOTAS_NU_NOTA_COMP2", "NOTAS_NU_NOTA_COMP3", "NOTAS_NU_NOTA_COMP4", "NOTAS_NU_NOTA_COMP5", "NOTAS_NU_NOTA_MEDIA", "NOTAS_NU_NOTA_CONCEITO", "NOTAS_NU_ACERTOS_CN", "NOTAS_NU_ACERTOS_CH", "NOTAS_NU_ACERTOS_LC", "NOTAS_NU_ACERTOS_MT", "NOTAS_NU_ACERTOS_TOTAL", "NOTAS_NU_ACERTOS_MEDIO", "PARTICIPANTE_TP_LINGUA", "PARTICIPANTE_RENDA_PER_CAPITA", "HABESTUDO_ST_MAT_PERC_APR", "HABESTUDO_GES_TEMP_PLAN_EST", "HABESTUDO_PRAT_EST_PES", "HABESTUDO_TECN_TP_ACES", "HABESTUDO_PROB_ROT_EST", "HABESTUDO_DIF_INFR", "HABESTUDO_AJUD_TERC", "HABESTUDO_AVAL_PROP_EXP")
FROM STDIN
WITH (DELIMITER ',',
ENCODING 'utf8',
FORMAT CSV,
HEADER);
'''


with db_connection() as conn, conn.cursor() as cursor:
    cursor.execute(filtered_aggregated_data_creation_query)
    conn.commit()
    
    with open('../filtered_aggregated_df.csv', encoding='utf8') as filtered_aggregated_csv_file:
        cursor.copy_expert(filtered_aggregated_csv_query, filtered_aggregated_csv_file)
        conn.commit()

#### Selecionando variáveis

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.filtered_aggregated_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''
    
    cursor.execute(query)
    filtered_aggregated_data = cursor.fetchall()
    filtered_aggregated_columns = tuple(desc[0] for desc in cursor.description)


filtered_aggregated_df = pd.DataFrame(filtered_aggregated_data, columns=filtered_aggregated_columns)
filtered_aggregated_df

**Pré-processamento**

**nominal:** 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_REGIAO_ESCOLA'

**numerical:** 'NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_ACERTOS_TOTAL', 'NOTAS_NU_ACERTOS_MEDIO', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_RENDA_PER_CAPITA', 'HABESTUDO_ST_MAT_PERC_APR', 'HABESTUDO_GES_TEMP_PLAN_EST', 'HABESTUDO_PRAT_EST_PES', 'HABESTUDO_TECN_TP_ACES', 'HABESTUDO_PROB_ROT_EST', 'HABESTUDO_DIF_INFR', 'HABESTUDO_AJUD_TERC', 'HABESTUDO_AVAL_PROP_EXP'

In [ ]:
nominal_features = list(set(filtered_aggregated_df.select_dtypes(include='object').columns.values))
numerical_features = list(set(filtered_aggregated_df.select_dtypes(exclude='object').columns.values))

In [ ]:
nominal_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
])

preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, nominal_features),
], remainder='passthrough')

In [ ]:
X = filtered_aggregated_df.drop(columns=['NU_INSCRICAO', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO'])
y_numerical = filtered_aggregated_df['NOTAS_NU_NOTA_MEDIA']
y_categorical = filtered_aggregated_df['NOTAS_NU_NOTA_CONCEITO']

X_transformed = preprocessor.fit_transform(X)
y_numerical_transformed = y_numerical
y_categorical_transformed = y_categorical

In [ ]:
preprocessor_feature_names = preprocessor.get_feature_names_out()
preprocessor_feature_names = [re.sub(r'.+__', '', item) for item in preprocessor_feature_names]

In [ ]:
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor_feature_names)
X_transformed

**Avaliando correlação**

In [ ]:
def plot_corr(ax, train_data, title):
    corr = train_data.corr(method='spearman')
    cmap = sns.diverging_palette(255, 255, sep=1, as_cmap=True)
    mask = np.triu(np.ones_like(corr, dtype=bool))

    graph = sns.heatmap(corr, annot=True, vmin=-1, vmax=1, fmt='.1%', cmap=cmap, mask=mask, ax=ax)
    graph.set_title(title, pad=10)

In [ ]:
fig, ax = plt.subplots(figsize=(32, 16), constrained_layout=True)

plot_corr(ax, X_transformed, 'Mapa de correlação entre variáveis')
plt.show()

In [ ]:
columns_to_drop = ['PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'PARTICIPANTE_TP_SEXO_M', 'NOTAS_NU_ACERTOS_MEDIO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_ACERTOS_TOTAL', 'PARTICIPANTE_TP_LINGUA']

X_transformed_dropped = X_transformed.drop(columns=columns_to_drop)
X_transformed_dropped

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10), constrained_layout=True)

plot_corr(ax, X_transformed_dropped, 'Mapa de correlação entre variáveis')
plt.show()

*Conclusões:*
- Linguagens e Humanas são as áreas que mais influenciam no número de acertos total;
- A competência 5 é a que mais influencia na nota de Redação;
- As categorias Gestão de Tempo e Planejamento de Estudos e Práticas de Estudo e Pesquisa são as mais relacionadas.

**Avaliando importância das variáveis**

In [ ]:
X_numerical_train, X_numerical_test, y_numerical_train, y_numerical_test = train_test_split(X_transformed_dropped, y_numerical_transformed, test_size=0.3, random_state=SEED)

X_categorical_train, X_categorical_test, y_categorical_train, y_categorical_test = train_test_split(X_transformed_dropped, y_categorical_transformed, test_size=0.3, random_state=SEED)

In [ ]:
# accelerating sklearn
patch_sklearn()

Classificador

In [ ]:
randomforest_classifier_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=SEED)),
])

In [ ]:
randomforest_classifier_pipe.fit(X_categorical_train, y_categorical_train)

In [ ]:
randomforest_classifier_accuracy_scores = cross_val_score(randomforest_classifier_pipe, X_transformed_dropped, y_categorical_transformed, cv=5, scoring='balanced_accuracy')
randomforest_classifier_accuracy_scores.mean()

In [ ]:
randomforest_classifier_precision_scores = cross_val_score(randomforest_classifier_pipe, X_transformed_dropped, y_categorical_transformed, cv=5, scoring='precision_weighted')
randomforest_classifier_precision_scores.mean()

In [ ]:
randomforest_classifier_recall_scores = cross_val_score(randomforest_classifier_pipe, X_transformed_dropped, y_categorical_transformed, cv=5, scoring='recall_weighted')
randomforest_classifier_recall_scores.mean()

In [ ]:
randomforest_classifier_f1_scores = cross_val_score(randomforest_classifier_pipe, X_transformed_dropped, y_categorical_transformed, cv=5, scoring='f1_weighted')
randomforest_classifier_f1_scores.mean()

In [ ]:
ConfusionMatrixDisplay.from_estimator(randomforest_classifier_pipe, X_transformed_dropped, y_categorical_transformed, cmap='Blues')
plt.show()

In [ ]:
randomforest_classifier_feature_importances = pd.Series(randomforest_classifier_pipe['classifier'].feature_importances_, index=X_categorical_train.columns)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.sort_values(ascending=False)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_classifier_feature_importances

Regressor

In [ ]:
randomforest_regressor_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=SEED)),
])

In [ ]:
randomforest_regressor_pipe.fit(X_numerical_train, y_numerical_train)

In [ ]:
randomforest_regressor_mae_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed_dropped, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_error')
randomforest_regressor_mae_scores.mean()

In [ ]:
randomforest_regressor_rmse_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed_dropped, y_numerical_transformed, cv=5, scoring='neg_root_mean_squared_error')
randomforest_regressor_rmse_scores.mean()

In [ ]:
randomforest_regressor_r2_scores = cross_val_score(randomforest_regressor_pipe, X_transformed_dropped, y_numerical_transformed, cv=5, scoring='r2')
randomforest_regressor_r2_scores.mean()

In [ ]:
randomforest_regressor_mape_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed_dropped, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_percentage_error')
randomforest_regressor_mape_scores.mean()

In [ ]:
randomforest_regressor_feature_importances = pd.Series(randomforest_regressor_pipe['regressor'].feature_importances_, index=X_numerical_train.columns)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.sort_values(ascending=False)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_regressor_feature_importances

In [ ]:
# decelerating sklearn
unpatch_sklearn()

**Utilizar hábitos de estudo mais significativos**

In [ ]:
def get_interest_hab_estudo(filtered_df: pd.DataFrame, full_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Substitui as colunas de categoria dos hábitos de estudo mais significativas e deleta as não utilizadas.

    :params:
    - filtered_df: DataFrame previamente tratado com todos os dados para substituição.
    - full_df: DataFrame completo para extração das colunas individuais de hábitos de estudo.

    Returns a DataFrame.
    '''

    filtered_df = filtered_df.drop(columns=['HABESTUDO_ST_MAT_PERC_APR', 'HABESTUDO_TECN_TP_ACES', 'HABESTUDO_DIF_INFR', 'HABESTUDO_AJUD_TERC', 'HABESTUDO_AVAL_PROP_EXP'])


    hab_estudo_transformation_dict = {
        'HABESTUDO_GES_TEMP_PLAN_EST': {
            'HABESTUDO_Q005': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q006': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q007': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q008': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q021': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q022': ('A', 'B', 'C', 'D'),
            },
        'HABESTUDO_PRAT_EST_PES': {
            'HABESTUDO_Q009': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q010': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q011': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q012': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q013': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q014': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q015': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q016': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q017': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q018': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q019': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q020': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q023': ('A', 'B', 'C', 'D'),
            'HABESTUDO_Q024': ('A', 'B', 'C', 'D'),
            },
        'HABESTUDO_PROB_ROT_EST': {
            'HABESTUDO_Q027': ('B', 'A'),
            'HABESTUDO_Q028A': ('B', 'A'),
            'HABESTUDO_Q028B': ('B', 'A'),
            'HABESTUDO_Q028C': ('B', 'A'),
            'HABESTUDO_Q028D': ('B', 'A'),
            'HABESTUDO_Q028E': ('B', 'A'),
            'HABESTUDO_Q028F': ('B', 'A'),
            'HABESTUDO_Q028G': ('B', 'A'),
            'HABESTUDO_Q028H': ('B', 'A'),
            'HABESTUDO_Q028I': ('B', 'A'),
            'HABESTUDO_Q028J': ('B', 'A'),
            'HABESTUDO_Q028K': ('B', 'A'),
            'HABESTUDO_Q028L': ('B', 'A'),
            'HABESTUDO_Q028M': ('B', 'A'),
            'HABESTUDO_Q028N': ('B', 'A'),
            'HABESTUDO_Q028O': ('B', 'A'),
            'HABESTUDO_Q028P': ('B', 'A'),
            'HABESTUDO_Q028Q': ('B', 'A'),
            'HABESTUDO_Q028R': ('B', 'A'),
            },
    }

    hab_estudo_columns = [column for columns_dict in hab_estudo_transformation_dict.values() for column in columns_dict.keys()]
    hab_estudo_columns = ['NU_INSCRICAO'] + hab_estudo_columns

    filtered_df = filtered_df.join(full_df[hab_estudo_columns].set_index('NU_INSCRICAO'), on='NU_INSCRICAO', how='inner')
    filtered_df = filtered_df.drop(columns=hab_estudo_transformation_dict.keys())

    return filtered_df

In [ ]:
complete_data = get_interest_hab_estudo(filtered_aggregated_df, aggregated_df)
complete_data

In [ ]:
columns_to_drop = ['PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_ACERTOS_MEDIO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_ACERTOS_TOTAL', 'PARTICIPANTE_TP_LINGUA']

complete_data = complete_data.drop(columns=columns_to_drop)
complete_data

In [ ]:
complete_data.to_csv('../complete_df.csv', index=False)

In [ ]:
complete_data_creation_query = '''
CREATE TABLE projeto_enem.complete_data (
	"NU_INSCRICAO" BIGINT PRIMARY KEY,
	"PARTICIPANTE_TP_FAIXA_ETARIA" INTEGER,
	"PARTICIPANTE_TP_SEXO" VARCHAR(1),
	"PARTICIPANTE_TP_COR_RACA" INTEGER,
	"PARTICIPANTE_TP_ESCOLA" INTEGER,
	"PARTICIPANTE_REGIAO_ESCOLA" VARCHAR(12),
	"NOTAS_NU_NOTA_MEDIA" FLOAT,
    "NOTAS_NU_NOTA_CONCEITO" INTEGER,
	"PARTICIPANTE_RENDA_PER_CAPITA" FLOAT,
	"HABESTUDO_Q005" VARCHAR(1),
	"HABESTUDO_Q006" VARCHAR(1),
	"HABESTUDO_Q007" VARCHAR(1),
	"HABESTUDO_Q008" VARCHAR(1),
	"HABESTUDO_Q021" VARCHAR(1),
	"HABESTUDO_Q022" VARCHAR(1),
	"HABESTUDO_Q009" VARCHAR(1),
	"HABESTUDO_Q010" VARCHAR(1),
	"HABESTUDO_Q011" VARCHAR(1),
	"HABESTUDO_Q012" VARCHAR(1),
	"HABESTUDO_Q013" VARCHAR(1),
	"HABESTUDO_Q014" VARCHAR(1),
	"HABESTUDO_Q015" VARCHAR(1),
	"HABESTUDO_Q016" VARCHAR(1),
	"HABESTUDO_Q017" VARCHAR(1),
	"HABESTUDO_Q018" VARCHAR(1),
	"HABESTUDO_Q019" VARCHAR(1),
	"HABESTUDO_Q020" VARCHAR(1),
	"HABESTUDO_Q023" VARCHAR(1),
	"HABESTUDO_Q024" VARCHAR(1),
	"HABESTUDO_Q027" VARCHAR(1),
	"HABESTUDO_Q028A" VARCHAR(1),
	"HABESTUDO_Q028B" VARCHAR(1),
	"HABESTUDO_Q028C" VARCHAR(1),
	"HABESTUDO_Q028D" VARCHAR(1),
	"HABESTUDO_Q028E" VARCHAR(1),
	"HABESTUDO_Q028F" VARCHAR(1),
	"HABESTUDO_Q028G" VARCHAR(1),
	"HABESTUDO_Q028H" VARCHAR(1),
	"HABESTUDO_Q028I" VARCHAR(1),
	"HABESTUDO_Q028J" VARCHAR(1),
	"HABESTUDO_Q028K" VARCHAR(1),
	"HABESTUDO_Q028L" VARCHAR(1),
	"HABESTUDO_Q028M" VARCHAR(1),
	"HABESTUDO_Q028N" VARCHAR(1),
	"HABESTUDO_Q028O" VARCHAR(1),
	"HABESTUDO_Q028P" VARCHAR(1),
	"HABESTUDO_Q028Q" VARCHAR(1),
	"HABESTUDO_Q028R" VARCHAR(1)
);
'''

complete_data_csv_query = '''
COPY projeto_enem.complete_data("NU_INSCRICAO", "PARTICIPANTE_TP_FAIXA_ETARIA", "PARTICIPANTE_TP_SEXO", "PARTICIPANTE_TP_COR_RACA", "PARTICIPANTE_TP_ESCOLA", "PARTICIPANTE_REGIAO_ESCOLA", "NOTAS_NU_NOTA_MEDIA", "NOTAS_NU_NOTA_CONCEITO", "PARTICIPANTE_RENDA_PER_CAPITA", "HABESTUDO_Q005", "HABESTUDO_Q006", "HABESTUDO_Q007", "HABESTUDO_Q008", "HABESTUDO_Q021", "HABESTUDO_Q022", "HABESTUDO_Q009", "HABESTUDO_Q010", "HABESTUDO_Q011", "HABESTUDO_Q012", "HABESTUDO_Q013", "HABESTUDO_Q014", "HABESTUDO_Q015", "HABESTUDO_Q016", "HABESTUDO_Q017", "HABESTUDO_Q018", "HABESTUDO_Q019", "HABESTUDO_Q020", "HABESTUDO_Q023", "HABESTUDO_Q024", "HABESTUDO_Q027", "HABESTUDO_Q028A", "HABESTUDO_Q028B", "HABESTUDO_Q028C", "HABESTUDO_Q028D", "HABESTUDO_Q028E", "HABESTUDO_Q028F", "HABESTUDO_Q028G", "HABESTUDO_Q028H", "HABESTUDO_Q028I", "HABESTUDO_Q028J", "HABESTUDO_Q028K", "HABESTUDO_Q028L", "HABESTUDO_Q028M", "HABESTUDO_Q028N", "HABESTUDO_Q028O", "HABESTUDO_Q028P", "HABESTUDO_Q028Q", "HABESTUDO_Q028R")
FROM STDIN
WITH (DELIMITER ',',
ENCODING 'utf8',
FORMAT CSV,
HEADER);
'''


with db_connection() as conn, conn.cursor() as cursor:
    cursor.execute(complete_data_creation_query)
    conn.commit()
    
    with open('../complete_df.csv', encoding='utf8') as complete_data_csv_file:
        cursor.copy_expert(complete_data_csv_query, complete_data_csv_file)
        conn.commit()

**Analisando importância das novas variáveis**

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.complete_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''

    cursor.execute(query)
    complete_data_fetched = cursor.fetchall()
    complete_data_columns = tuple(desc[0] for desc in cursor.description)


complete_data = pd.DataFrame(complete_data_fetched, columns=complete_data_columns)
complete_data

**ordinal:** 'HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R'

**nominal:** 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_REGIAO_ESCOLA'

**numerical:** 'NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_ACERTOS_TOTAL', 'NOTAS_NU_ACERTOS_MEDIO', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_RENDA_PER_CAPITA'

In [ ]:
ordinal_features = ['HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R']
nominal_features = list(set(complete_data.select_dtypes(include='object').columns.values) - set(ordinal_features))
numerical_features = list(set(complete_data.select_dtypes(exclude='object').columns.values) - set(ordinal_features))

In [ ]:
ordinal_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(categories=[
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q005
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q006
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q007
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q008
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q021
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q022
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q009
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q010
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q011
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q012
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q013
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q014
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q015
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q016
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q017
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q018
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q019
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q020
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q023
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q024
        ('B', 'A'),   # HABESTUDO_Q027
        ('B', 'A'),   # HABESTUDO_Q028A
        ('B', 'A'),   # HABESTUDO_Q028B
        ('B', 'A'),   # HABESTUDO_Q028C
        ('B', 'A'),   # HABESTUDO_Q028D
        ('B', 'A'),   # HABESTUDO_Q028E
        ('B', 'A'),   # HABESTUDO_Q028F
        ('B', 'A'),   # HABESTUDO_Q028G
        ('B', 'A'),   # HABESTUDO_Q028H
        ('B', 'A'),   # HABESTUDO_Q028I
        ('B', 'A'),   # HABESTUDO_Q028J
        ('B', 'A'),   # HABESTUDO_Q028K
        ('B', 'A'),   # HABESTUDO_Q028L
        ('B', 'A'),   # HABESTUDO_Q028M
        ('B', 'A'),   # HABESTUDO_Q028N
        ('B', 'A'),   # HABESTUDO_Q028O
        ('B', 'A'),   # HABESTUDO_Q028P
        ('B', 'A'),   # HABESTUDO_Q028Q
        ('B', 'A'),   # HABESTUDO_Q028R
    ])),
])

nominal_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
])

preprocessor = ColumnTransformer(transformers=[
    ('ordinal', ordinal_transformer, ordinal_features),
    ('nominal', nominal_transformer, nominal_features),
], remainder='passthrough')

In [ ]:
X = complete_data.drop(columns=['NU_INSCRICAO', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO'])

X_transformed = preprocessor.fit_transform(X)
y_numerical_transformed = complete_data['NOTAS_NU_NOTA_MEDIA']
y_categorical_transformed = complete_data['NOTAS_NU_NOTA_CONCEITO']

In [ ]:
preprocessor_feature_names = preprocessor.get_feature_names_out()
preprocessor_feature_names = [re.sub(r'.+__', '', item) for item in preprocessor_feature_names]

In [ ]:
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor_feature_names)
X_transformed = X_transformed.drop(columns='PARTICIPANTE_TP_SEXO_M').reset_index(drop=True)
X_transformed

In [ ]:
X_numerical_train, X_numerical_test, y_numerical_train, y_numerical_test = train_test_split(X_transformed, y_numerical_transformed, test_size=0.3, random_state=SEED)

X_categorical_train, X_categorical_test, y_categorical_train, y_categorical_test = train_test_split(X_transformed, y_categorical_transformed, test_size=0.3, random_state=SEED)

In [ ]:
# accelerating sklearn
patch_sklearn()

Classificador

In [ ]:
randomforest_classifier_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=SEED)),
])

In [ ]:
randomforest_classifier_pipe.fit(X_categorical_train, y_categorical_train)

In [ ]:
randomforest_classifier_accuracy_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='balanced_accuracy')
randomforest_classifier_accuracy_scores.mean()

In [ ]:
randomforest_classifier_precision_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='precision_weighted')
randomforest_classifier_precision_scores.mean()

In [ ]:
randomforest_classifier_recall_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='recall_weighted')
randomforest_classifier_recall_scores.mean()

In [ ]:
randomforest_classifier_f1_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='f1_weighted')
randomforest_classifier_f1_scores.mean()

In [ ]:
ConfusionMatrixDisplay.from_estimator(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cmap='Blues')
plt.show()

In [ ]:
randomforest_classifier_feature_importances = pd.Series(randomforest_classifier_pipe['classifier'].feature_importances_, index=X_categorical_train.columns)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.sort_values(ascending=False)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_classifier_feature_importances

Regressor

In [ ]:
randomforest_regressor_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=SEED)),
])

In [ ]:
randomforest_regressor_pipe.fit(X_numerical_train, y_numerical_train)

In [ ]:
randomforest_regressor_mae_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_error')
randomforest_regressor_mae_scores.mean()

In [ ]:
randomforest_regressor_rmse_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_root_mean_squared_error')
randomforest_regressor_rmse_scores.mean()

In [ ]:
randomforest_regressor_r2_scores = cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='r2')
randomforest_regressor_r2_scores.mean()

In [ ]:
randomforest_regressor_mape_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_percentage_error')
randomforest_regressor_mape_scores.mean()

In [ ]:
randomforest_regressor_feature_importances = pd.Series(randomforest_regressor_pipe['regressor'].feature_importances_, index=X_numerical_train.columns)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.sort_values(ascending=False)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_regressor_feature_importances

In [ ]:
# decelerating sklearn
unpatch_sklearn()

In [ ]:
randomforest_feature_importances = pd.concat([pd.DataFrame(randomforest_classifier_feature_importances, columns=['classifier_importance']).reset_index(names='classifier_column'), pd.DataFrame(randomforest_regressor_feature_importances, columns=['regressor_importance']).reset_index(names='regressor_column')], axis=1).sort_values(['classifier_importance', 'regressor_importance'], ascending=False)
randomforest_feature_importances

In [ ]:
def get_hab_estudo_correspondent_column_names(df: pd.DataFrame, changing_columns: list) -> pd.DataFrame:
    '''
    Substitui as colunas de categoria dos hábitos de estudo pelo seu significado.

    :params:
    - df: DataFrame previamente tratado com todos os dados para substituição.
    - changing_columns: colunas com dados para serem alterados.

    Returns a DataFrame.
    '''

    hab_estudo_column_correspondent_names = {
        'HABESTUDO_Q005': 'Organizei cronograma de estudos com tempos mais longos e mais curtos para estudar de acordo com a dificuldade das matérias',
        'HABESTUDO_Q006': 'Reservei tempos mais longos e mais curtos para estudar de acordo com a dificuldade das matérias',
        'HABESTUDO_Q007': 'Organizei material para ser estudado',
        'HABESTUDO_Q008': 'Eu me dediquei aos horários programados de estudo de acordo com a dificuldade das matérias',
        'HABESTUDO_Q021': 'Entrei nas aulas online por videoconferência sem atraso da minha parte',
        'HABESTUDO_Q022': 'Assisti todas as aulas online nas datas programadas para estudo',
        'HABESTUDO_Q009': 'Li os textos indicados em cada matéria antes de assistir as aulas ou videoaulas sobre o assunto dos textos',
        'HABESTUDO_Q010': 'Resumi os textos das matérias, destacando as partes mais importantes',
        'HABESTUDO_Q011': 'Resumi as videoaulas ou os podcasts, destacando as partes mais importantes',
        'HABESTUDO_Q012': 'Fiz as atividades das matérias para fixação de conteúdo',
        'HABESTUDO_Q013': 'Fiz atividades avaliativas, inclusive simulados, para verificar o quanto aprendi durante a pandemia',
        'HABESTUDO_Q014': 'Aproveitei o tempo das aulas online ou atividades de reforço, sem desperdiçá-lo com distrações',
        'HABESTUDO_Q015': 'Anotei as explicações obtidas em videoaulas ou podcasts das matérias',
        'HABESTUDO_Q016': 'Anotei as informações que obtive ao assistir vídeos complementares de assuntos do meu interesse',
        'HABESTUDO_Q017': 'Destaquei as dúvidas que tive ao ler os textos das disciplinas para esclarecer com os professores',
        'HABESTUDO_Q018': 'Estruturei as principais ideias para produzir redações',
        'HABESTUDO_Q019': 'Treinei redação',
        'HABESTUDO_Q020': 'Participei de fóruns de discussão por matéria para tirar dúvidas',
        'HABESTUDO_Q023': 'Revisei as anotações das aulas, os resumos e anotações dos demais materiais que li ou assisti',
        'HABESTUDO_Q024': 'Reassisti as videoaulas e os podcasts das matérias',
        'HABESTUDO_Q027': 'Você vivenciou problemas em sua rotina para estudar ou manter-se informado(a) durante a pandemia?',
        'HABESTUDO_Q028A': 'Eu me senti desestimulado(a) por não ter colegas com quem interagir sobre o que eu estava estudando',
        'HABESTUDO_Q028B': 'Tive dificuldade de compreender os conteúdos por falta de explicações de um professor em tempo real',
        'HABESTUDO_Q028C': 'Fiquei por muito tempo diante das telas, sem pequenos intervalos para descansar',
        'HABESTUDO_Q028D': 'Reduzi a prática de atividades físicas',
        'HABESTUDO_Q028E': 'Dormi por menos tempo',
        'HABESTUDO_Q028F': 'Tive episódios de insônia',
        'HABESTUDO_Q028G': 'Senti dificuldade em manter a motivação para estudar por minha conta',
        'HABESTUDO_Q028H': 'Senti dificuldade em me motivar a cumprir meu cronograma',
        'HABESTUDO_Q028I': 'Senti medo de não conseguir aprender os conteúdos',
        'HABESTUDO_Q028J': 'Senti ansiedade devido ao isolamento social e ao receio de contágio da doença',
        'HABESTUDO_Q028K': 'Dediquei tempo para ajudar algum familiar a estudar em casa',
        'HABESTUDO_Q028L': 'Precisei ficar em repouso porque tive Covid com sintomas',
        'HABESTUDO_Q028M': 'Precisei ficar em repouso porque tive outra doença',
        'HABESTUDO_Q028N': 'Precisei cuidar de algum familiar que teve Covid com sintomas',
        'HABESTUDO_Q028O': 'Precisei cuidar de algum familiar que teve outra doença',
        'HABESTUDO_Q028P': 'Tive que cuidar do(s) meu(s) irmão(s) menor(es) para meus pais trabalharem',
        'HABESTUDO_Q028Q': 'Tive que começar a trabalhar em casa para obter renda',
        'HABESTUDO_Q028R': 'Tive que trabalhar em serviço considerado essencial durante a pandemia',
    }

    for column in changing_columns:
        df.loc[:, column] = df[column].replace(hab_estudo_column_correspondent_names)

    return df

In [ ]:
randomforest_feature_importances = get_hab_estudo_correspondent_column_names(randomforest_feature_importances, ['classifier_column', 'regressor_column'])
randomforest_feature_importances

In [ ]:
randomforest_feature_importances[['regressor_column', 'regressor_importance']]

**Analisando importância em escola particular**

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.complete_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''

    cursor.execute(query)
    complete_data_fetched = cursor.fetchall()
    complete_data_columns = tuple(desc[0] for desc in cursor.description)


complete_data = pd.DataFrame(complete_data_fetched, columns=complete_data_columns)
complete_data

In [ ]:
complete_data_private = complete_data[complete_data['PARTICIPANTE_TP_ESCOLA'] == 3]
complete_data_private = complete_data_private.drop(columns='PARTICIPANTE_TP_ESCOLA')
complete_data_private

**ordinal:** 'HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R'

**nominal:** 'PARTICIPANTE_TP_SEXO', 'PARTICIPANTE_REGIAO_ESCOLA'

**numerical:** 'NU_INSCRICAO', 'PARTICIPANTE_TP_FAIXA_ETARIA', 'PARTICIPANTE_TP_COR_RACA', 'PARTICIPANTE_TP_ST_CONCLUSAO', 'PARTICIPANTE_TP_ANO_CONCLUIU', 'PARTICIPANTE_TP_ESCOLA', 'PARTICIPANTE_IN_TREINEIRO', 'PARTICIPANTE_TP_PRESENCA_CN', 'PARTICIPANTE_TP_PRESENCA_CH', 'PARTICIPANTE_TP_PRESENCA_LC', 'PARTICIPANTE_TP_PRESENCA_MT', 'NOTAS_NU_NOTA_CN', 'NOTAS_NU_NOTA_CH', 'NOTAS_NU_NOTA_LC', 'NOTAS_NU_NOTA_MT', 'NOTAS_NU_NOTA_REDACAO', 'PARTICIPANTE_TP_STATUS_REDACAO', 'NOTAS_NU_NOTA_COMP1', 'NOTAS_NU_NOTA_COMP2', 'NOTAS_NU_NOTA_COMP3', 'NOTAS_NU_NOTA_COMP4', 'NOTAS_NU_NOTA_COMP5', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO', 'NOTAS_NU_ACERTOS_CN', 'NOTAS_NU_ACERTOS_CH', 'NOTAS_NU_ACERTOS_LC', 'NOTAS_NU_ACERTOS_MT', 'NOTAS_NU_ACERTOS_TOTAL', 'NOTAS_NU_ACERTOS_MEDIO', 'PARTICIPANTE_TP_LINGUA', 'PARTICIPANTE_RENDA_PER_CAPITA'

In [ ]:
ordinal_features = ['HABESTUDO_Q005', 'HABESTUDO_Q006', 'HABESTUDO_Q007', 'HABESTUDO_Q008', 'HABESTUDO_Q021', 'HABESTUDO_Q022', 'HABESTUDO_Q009', 'HABESTUDO_Q010', 'HABESTUDO_Q011', 'HABESTUDO_Q012', 'HABESTUDO_Q013', 'HABESTUDO_Q014', 'HABESTUDO_Q015', 'HABESTUDO_Q016', 'HABESTUDO_Q017', 'HABESTUDO_Q018', 'HABESTUDO_Q019', 'HABESTUDO_Q020', 'HABESTUDO_Q023', 'HABESTUDO_Q024', 'HABESTUDO_Q027', 'HABESTUDO_Q028A', 'HABESTUDO_Q028B', 'HABESTUDO_Q028C', 'HABESTUDO_Q028D', 'HABESTUDO_Q028E', 'HABESTUDO_Q028F', 'HABESTUDO_Q028G', 'HABESTUDO_Q028H', 'HABESTUDO_Q028I', 'HABESTUDO_Q028J', 'HABESTUDO_Q028K', 'HABESTUDO_Q028L', 'HABESTUDO_Q028M', 'HABESTUDO_Q028N', 'HABESTUDO_Q028O', 'HABESTUDO_Q028P', 'HABESTUDO_Q028Q', 'HABESTUDO_Q028R']
nominal_features = list(set(complete_data.select_dtypes(include='object').columns.values) - set(ordinal_features))
numerical_features = list(set(complete_data.select_dtypes(exclude='object').columns.values) - set(ordinal_features))

In [ ]:
ordinal_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(categories=[
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q005
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q006
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q007
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q008
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q021
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q022
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q009
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q010
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q011
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q012
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q013
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q014
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q015
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q016
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q017
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q018
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q019
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q020
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q023
        ('A', 'B', 'C', 'D'),   # HABESTUDO_Q024
        ('B', 'A'),   # HABESTUDO_Q027
        ('B', 'A'),   # HABESTUDO_Q028A
        ('B', 'A'),   # HABESTUDO_Q028B
        ('B', 'A'),   # HABESTUDO_Q028C
        ('B', 'A'),   # HABESTUDO_Q028D
        ('B', 'A'),   # HABESTUDO_Q028E
        ('B', 'A'),   # HABESTUDO_Q028F
        ('B', 'A'),   # HABESTUDO_Q028G
        ('B', 'A'),   # HABESTUDO_Q028H
        ('B', 'A'),   # HABESTUDO_Q028I
        ('B', 'A'),   # HABESTUDO_Q028J
        ('B', 'A'),   # HABESTUDO_Q028K
        ('B', 'A'),   # HABESTUDO_Q028L
        ('B', 'A'),   # HABESTUDO_Q028M
        ('B', 'A'),   # HABESTUDO_Q028N
        ('B', 'A'),   # HABESTUDO_Q028O
        ('B', 'A'),   # HABESTUDO_Q028P
        ('B', 'A'),   # HABESTUDO_Q028Q
        ('B', 'A'),   # HABESTUDO_Q028R
    ])),
])

nominal_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
])

preprocessor = ColumnTransformer(transformers=[
    ('ordinal', ordinal_transformer, ordinal_features),
    ('nominal', nominal_transformer, nominal_features),
], remainder='passthrough')

In [ ]:
X = complete_data_private.drop(columns=['NU_INSCRICAO', 'NOTAS_NU_NOTA_MEDIA', 'NOTAS_NU_NOTA_CONCEITO'])

X_transformed = preprocessor.fit_transform(X)
y_numerical_transformed = complete_data_private['NOTAS_NU_NOTA_MEDIA']
y_categorical_transformed = complete_data_private['NOTAS_NU_NOTA_CONCEITO']

In [ ]:
preprocessor_feature_names = preprocessor.get_feature_names_out()
preprocessor_feature_names = [re.sub(r'.+__', '', item) for item in preprocessor_feature_names]

In [ ]:
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor_feature_names)
X_transformed = X_transformed.drop(columns='PARTICIPANTE_TP_SEXO_M').reset_index(drop=True)
X_transformed

In [ ]:
X_numerical_train, X_numerical_test, y_numerical_train, y_numerical_test = train_test_split(X_transformed, y_numerical_transformed, test_size=0.3, random_state=SEED)

X_categorical_train, X_categorical_test, y_categorical_train, y_categorical_test = train_test_split(X_transformed, y_categorical_transformed, test_size=0.3, random_state=SEED)

In [ ]:
# accelerating sklearn
patch_sklearn()

Classificador

In [ ]:
randomforest_classifier_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=SEED)),
])

In [ ]:
randomforest_classifier_pipe.fit(X_categorical_train, y_categorical_train)

In [ ]:
randomforest_classifier_accuracy_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='balanced_accuracy')
randomforest_classifier_accuracy_scores.mean()

In [ ]:
randomforest_classifier_precision_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='precision_weighted')
randomforest_classifier_precision_scores.mean()

In [ ]:
randomforest_classifier_recall_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='recall_weighted')
randomforest_classifier_recall_scores.mean()

In [ ]:
randomforest_classifier_f1_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='f1_weighted')
randomforest_classifier_f1_scores.mean()

In [ ]:
ConfusionMatrixDisplay.from_estimator(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cmap='Blues')
plt.show()

In [ ]:
randomforest_classifier_feature_importances = pd.Series(randomforest_classifier_pipe['classifier'].feature_importances_, index=X_categorical_train.columns)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.sort_values(ascending=False)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_classifier_feature_importances

Regressor

In [ ]:
randomforest_regressor_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=SEED)),
])

In [ ]:
randomforest_regressor_pipe.fit(X_numerical_train, y_numerical_train)

In [ ]:
randomforest_regressor_mae_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_error')
randomforest_regressor_mae_scores.mean()

In [ ]:
randomforest_regressor_rmse_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_root_mean_squared_error')
randomforest_regressor_rmse_scores.mean()

In [ ]:
randomforest_regressor_r2_scores = cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='r2')
randomforest_regressor_r2_scores.mean()

In [ ]:
randomforest_regressor_mape_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_percentage_error')
randomforest_regressor_mape_scores.mean()

In [ ]:
randomforest_regressor_feature_importances = pd.Series(randomforest_regressor_pipe['regressor'].feature_importances_, index=X_numerical_train.columns)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.sort_values(ascending=False)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_regressor_feature_importances

In [ ]:
# decelerating sklearn
unpatch_sklearn()

In [ ]:
randomforest_feature_importances = pd.concat([pd.DataFrame(randomforest_classifier_feature_importances, columns=['classifier_importance']).reset_index(names='classifier_column'), pd.DataFrame(randomforest_regressor_feature_importances, columns=['regressor_importance']).reset_index(names='regressor_column')], axis=1).sort_values(['classifier_importance', 'regressor_importance'], ascending=False)
randomforest_feature_importances

In [ ]:
def get_hab_estudo_correspondent_column_names(df: pd.DataFrame, changing_columns: list) -> pd.DataFrame:
    '''
    Substitui as colunas de categoria dos hábitos de estudo pelo seu significado.

    :params:
    - df: DataFrame previamente tratado com todos os dados para substituição.
    - changing_columns: colunas com dados para serem alterados.

    Returns a DataFrame.
    '''

    hab_estudo_column_correspondent_names = {
        'HABESTUDO_Q005': 'Organizei cronograma de estudos com tempos mais longos e mais curtos para estudar de acordo com a dificuldade das matérias',
        'HABESTUDO_Q006': 'Reservei tempos mais longos e mais curtos para estudar de acordo com a dificuldade das matérias',
        'HABESTUDO_Q007': 'Organizei material para ser estudado',
        'HABESTUDO_Q008': 'Eu me dediquei aos horários programados de estudo de acordo com a dificuldade das matérias',
        'HABESTUDO_Q021': 'Entrei nas aulas online por videoconferência sem atraso da minha parte',
        'HABESTUDO_Q022': 'Assisti todas as aulas online nas datas programadas para estudo',
        'HABESTUDO_Q009': 'Li os textos indicados em cada matéria antes de assistir as aulas ou videoaulas sobre o assunto dos textos',
        'HABESTUDO_Q010': 'Resumi os textos das matérias, destacando as partes mais importantes',
        'HABESTUDO_Q011': 'Resumi as videoaulas ou os podcasts, destacando as partes mais importantes',
        'HABESTUDO_Q012': 'Fiz as atividades das matérias para fixação de conteúdo',
        'HABESTUDO_Q013': 'Fiz atividades avaliativas, inclusive simulados, para verificar o quanto aprendi durante a pandemia',
        'HABESTUDO_Q014': 'Aproveitei o tempo das aulas online ou atividades de reforço, sem desperdiçá-lo com distrações',
        'HABESTUDO_Q015': 'Anotei as explicações obtidas em videoaulas ou podcasts das matérias',
        'HABESTUDO_Q016': 'Anotei as informações que obtive ao assistir vídeos complementares de assuntos do meu interesse',
        'HABESTUDO_Q017': 'Destaquei as dúvidas que tive ao ler os textos das disciplinas para esclarecer com os professores',
        'HABESTUDO_Q018': 'Estruturei as principais ideias para produzir redações',
        'HABESTUDO_Q019': 'Treinei redação',
        'HABESTUDO_Q020': 'Participei de fóruns de discussão por matéria para tirar dúvidas',
        'HABESTUDO_Q023': 'Revisei as anotações das aulas, os resumos e anotações dos demais materiais que li ou assisti',
        'HABESTUDO_Q024': 'Reassisti as videoaulas e os podcasts das matérias',
        'HABESTUDO_Q027': 'Você vivenciou problemas em sua rotina para estudar ou manter-se informado(a) durante a pandemia?',
        'HABESTUDO_Q028A': 'Eu me senti desestimulado(a) por não ter colegas com quem interagir sobre o que eu estava estudando',
        'HABESTUDO_Q028B': 'Tive dificuldade de compreender os conteúdos por falta de explicações de um professor em tempo real',
        'HABESTUDO_Q028C': 'Fiquei por muito tempo diante das telas, sem pequenos intervalos para descansar',
        'HABESTUDO_Q028D': 'Reduzi a prática de atividades físicas',
        'HABESTUDO_Q028E': 'Dormi por menos tempo',
        'HABESTUDO_Q028F': 'Tive episódios de insônia',
        'HABESTUDO_Q028G': 'Senti dificuldade em manter a motivação para estudar por minha conta',
        'HABESTUDO_Q028H': 'Senti dificuldade em me motivar a cumprir meu cronograma',
        'HABESTUDO_Q028I': 'Senti medo de não conseguir aprender os conteúdos',
        'HABESTUDO_Q028J': 'Senti ansiedade devido ao isolamento social e ao receio de contágio da doença',
        'HABESTUDO_Q028K': 'Dediquei tempo para ajudar algum familiar a estudar em casa',
        'HABESTUDO_Q028L': 'Precisei ficar em repouso porque tive Covid com sintomas',
        'HABESTUDO_Q028M': 'Precisei ficar em repouso porque tive outra doença',
        'HABESTUDO_Q028N': 'Precisei cuidar de algum familiar que teve Covid com sintomas',
        'HABESTUDO_Q028O': 'Precisei cuidar de algum familiar que teve outra doença',
        'HABESTUDO_Q028P': 'Tive que cuidar do(s) meu(s) irmão(s) menor(es) para meus pais trabalharem',
        'HABESTUDO_Q028Q': 'Tive que começar a trabalhar em casa para obter renda',
        'HABESTUDO_Q028R': 'Tive que trabalhar em serviço considerado essencial durante a pandemia',
    }

    for column in changing_columns:
        df.loc[:, column] = df[column].replace(hab_estudo_column_correspondent_names)

    return df

In [ ]:
randomforest_feature_importances = get_hab_estudo_correspondent_column_names(randomforest_feature_importances, ['classifier_column', 'regressor_column'])
randomforest_feature_importances

In [ ]:
randomforest_feature_importances[['regressor_column', 'regressor_importance']]

In [ ]:
complete_data_private.to_csv('../complete_private_df.csv', index=False)

In [ ]:
complete_private_data_creation_query = '''
CREATE TABLE projeto_enem.complete_private_data (
	"NU_INSCRICAO" BIGINT PRIMARY KEY,
	"PARTICIPANTE_TP_FAIXA_ETARIA" INTEGER,
	"PARTICIPANTE_TP_SEXO" VARCHAR(1),
	"PARTICIPANTE_TP_COR_RACA" INTEGER,
	"PARTICIPANTE_REGIAO_ESCOLA" VARCHAR(12),
	"NOTAS_NU_NOTA_MEDIA" FLOAT,
    "NOTAS_NU_NOTA_CONCEITO" INTEGER,
	"PARTICIPANTE_RENDA_PER_CAPITA" FLOAT,
	"HABESTUDO_Q005" VARCHAR(1),
	"HABESTUDO_Q006" VARCHAR(1),
	"HABESTUDO_Q007" VARCHAR(1),
	"HABESTUDO_Q008" VARCHAR(1),
	"HABESTUDO_Q021" VARCHAR(1),
	"HABESTUDO_Q022" VARCHAR(1),
	"HABESTUDO_Q009" VARCHAR(1),
	"HABESTUDO_Q010" VARCHAR(1),
	"HABESTUDO_Q011" VARCHAR(1),
	"HABESTUDO_Q012" VARCHAR(1),
	"HABESTUDO_Q013" VARCHAR(1),
	"HABESTUDO_Q014" VARCHAR(1),
	"HABESTUDO_Q015" VARCHAR(1),
	"HABESTUDO_Q016" VARCHAR(1),
	"HABESTUDO_Q017" VARCHAR(1),
	"HABESTUDO_Q018" VARCHAR(1),
	"HABESTUDO_Q019" VARCHAR(1),
	"HABESTUDO_Q020" VARCHAR(1),
	"HABESTUDO_Q023" VARCHAR(1),
	"HABESTUDO_Q024" VARCHAR(1),
	"HABESTUDO_Q027" VARCHAR(1),
	"HABESTUDO_Q028A" VARCHAR(1),
	"HABESTUDO_Q028B" VARCHAR(1),
	"HABESTUDO_Q028C" VARCHAR(1),
	"HABESTUDO_Q028D" VARCHAR(1),
	"HABESTUDO_Q028E" VARCHAR(1),
	"HABESTUDO_Q028F" VARCHAR(1),
	"HABESTUDO_Q028G" VARCHAR(1),
	"HABESTUDO_Q028H" VARCHAR(1),
	"HABESTUDO_Q028I" VARCHAR(1),
	"HABESTUDO_Q028J" VARCHAR(1),
	"HABESTUDO_Q028K" VARCHAR(1),
	"HABESTUDO_Q028L" VARCHAR(1),
	"HABESTUDO_Q028M" VARCHAR(1),
	"HABESTUDO_Q028N" VARCHAR(1),
	"HABESTUDO_Q028O" VARCHAR(1),
	"HABESTUDO_Q028P" VARCHAR(1),
	"HABESTUDO_Q028Q" VARCHAR(1),
	"HABESTUDO_Q028R" VARCHAR(1)
);
'''

complete_private_data_csv_query = '''
COPY projeto_enem.complete_private_data("NU_INSCRICAO", "PARTICIPANTE_TP_FAIXA_ETARIA", "PARTICIPANTE_TP_SEXO", "PARTICIPANTE_TP_COR_RACA", "PARTICIPANTE_REGIAO_ESCOLA", "NOTAS_NU_NOTA_MEDIA", "NOTAS_NU_NOTA_CONCEITO", "PARTICIPANTE_RENDA_PER_CAPITA", "HABESTUDO_Q005", "HABESTUDO_Q006", "HABESTUDO_Q007", "HABESTUDO_Q008", "HABESTUDO_Q021", "HABESTUDO_Q022", "HABESTUDO_Q009", "HABESTUDO_Q010", "HABESTUDO_Q011", "HABESTUDO_Q012", "HABESTUDO_Q013", "HABESTUDO_Q014", "HABESTUDO_Q015", "HABESTUDO_Q016", "HABESTUDO_Q017", "HABESTUDO_Q018", "HABESTUDO_Q019", "HABESTUDO_Q020", "HABESTUDO_Q023", "HABESTUDO_Q024", "HABESTUDO_Q027", "HABESTUDO_Q028A", "HABESTUDO_Q028B", "HABESTUDO_Q028C", "HABESTUDO_Q028D", "HABESTUDO_Q028E", "HABESTUDO_Q028F", "HABESTUDO_Q028G", "HABESTUDO_Q028H", "HABESTUDO_Q028I", "HABESTUDO_Q028J", "HABESTUDO_Q028K", "HABESTUDO_Q028L", "HABESTUDO_Q028M", "HABESTUDO_Q028N", "HABESTUDO_Q028O", "HABESTUDO_Q028P", "HABESTUDO_Q028Q", "HABESTUDO_Q028R")
FROM STDIN
WITH (DELIMITER ',',
ENCODING 'utf8',
FORMAT CSV,
HEADER);
'''


with db_connection() as conn, conn.cursor() as cursor:
    cursor.execute(complete_private_data_creation_query)
    conn.commit()
    
    with open('../complete_private_df.csv', encoding='utf8') as complete_private_data_csv_file:
        cursor.copy_expert(complete_private_data_csv_query, complete_private_data_csv_file)
        conn.commit()

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.complete_private_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''

    cursor.execute(query)
    complete_private_data_fetched = cursor.fetchall()
    complete_private_data_columns = tuple(desc[0] for desc in cursor.description)


complete_private_data = pd.DataFrame(complete_private_data_fetched, columns=complete_private_data_columns)
complete_private_data